1. Реализовать персональный топ - принимает на вход возраст и локацию, на выходе персональный топ - 1 балл
Персональный топ - это топ товаров по похожим возрасту/интересам/локации. Как сделать? Разбить на сегменты по выбраным признакам. Топ делать по книгам с хорошим средним рейтингом.

2. На основе метода кластеризации похожих пользователей построить рекомендации (Слайд 27) - 3 балла
Нужно топ-10 рекомендаций с самой высокой оценкой. Считаем среднюю оценку для каждой книги по кластеру и выводим топ-10 книг.

3. Совстречаемость - 3 балла
В совстречаемости также учитывать оценки. Вес пары книг встретившихся у пользователя - полусумма их оценок.

4. Коллаборативная фильтрация - 3 балла
Коллаборативную фильтрацию реализовывать как на слайде 51 презентации, посоветовав каждому пользователю топ-10 книг с самой высокой оценкой. Сделать рекомендации User-based и Item-based и сравнить.

Если совсем сложно - можно сделать как в семинарской части, поставив оценку "0", если рейтинг < 5 и "1" - в противном случае. Тогда максимум за это - 1 балл. Реализовать U2I и I2I рекомендации.

Примечание:
Так как пользователей много - можно зафиксировать несколько произвольных и для них уже составлять рекомендации

Работоспособность I2I можно проверять на известных книгах (Гарри Поттер, Властелин Колец, Интервью с вампиром, Код-Да-Винчи, Маленький Принц)

Рейтинг книг обязательно нужно учитывать

Не забываем также предобработать данные - выкинуть выбросы-пользователей и выбросы-книги.

In [ ]:
import numpy as np
import pandas as pd
import tqdm
import re

from io import BytesIO
from PIL import Image
import requests
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
import gensim
from scipy.sparse import coo_matrix, csr_matrix
from scipy.sparse.linalg import svds

In [ ]:
books = pd.read_csv("BX-Books.csv")

In [ ]:
books.dropna(inplace=True)

In [ ]:
interactions = pd.read_csv("BX-Book-Ratings.csv", sep=";", encoding = "ISO-8859-1")

In [ ]:
interactions = interactions[interactions["Book-Rating"] != 0]

In [ ]:
books_meets = interactions.groupby("ISBN")["User-ID"].count().reset_index().rename(columns={"User-ID": "user_num"})

In [ ]:
user_meets = interactions.groupby("User-ID")["ISBN"].count().reset_index().rename(columns={"ISBN": "books_num"})

In [ ]:
interactions = interactions.merge(books_meets, on=["ISBN"]).merge(user_meets, on=["User-ID"])

In [ ]:
interactions = interactions[(interactions["user_num"] > 5) &
                            (interactions["books_num"] > 5) &
                            (interactions["books_num"] < 200)]

In [ ]:
users = pd.read_csv('BX-Users.csv', delimiter=';', encoding = 'ISO-8859-1')

In [ ]:
interactions = interactions.merge(books[["ISBN", "Image-URL-M", "Book-Title"]].rename(
    columns={"Image-URL-M": "picture_url"}), on=["ISBN"])

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [ ]:
interactions["product_id"] = le.fit_transform(interactions["ISBN"])
interactions["vid"] = le.fit_transform(interactions["User-ID"])

In [ ]:
interactions.head()

,User-ID,ISBN,Book-Rating,user_num,books_num,picture_url,Book-Title,product_id,vid
0,86583,3404139178,9,8,18,http://images.amazon.com/images/P/3404139178.0...,Das Lacheln der Fortuna: Historischer Roman,10391,3445
1,132500,3404139178,10,8,43,http://images.amazon.com/images/P/3404139178.0...,Das Lacheln der Fortuna: Historischer Roman,10391,5248
2,66483,3404139178,10,8,83,http://images.amazon.com/images/P/3404139178.0...,Das Lacheln der Fortuna: Historischer Roman,10391,2622
3,276866,3404139178,9,8,11,http://images.amazon.com/images/P/3404139178.0...,Das Lacheln der Fortuna: Historischer Roman,10391,10886
4,106534,3404139178,6,8,6,http://images.amazon.com/images/P/3404139178.0...,Das Lacheln der Fortuna: Historischer Roman,10391,4228


In [ ]:
interactions['product_id'] = le.fit_transform(interactions['ISBN'])
interactions['vid'] = le.fit_transform(interactions['User-ID'])

In [ ]:
interactions.head()

,User-ID,ISBN,Book-Rating,user_num,books_num,picture_url,Book-Title,product_id,vid
0,86583,3404139178,9,8,18,http://images.amazon.com/images/P/3404139178.0...,Das Lacheln der Fortuna: Historischer Roman,10391,3445
1,132500,3404139178,10,8,43,http://images.amazon.com/images/P/3404139178.0...,Das Lacheln der Fortuna: Historischer Roman,10391,5248
2,66483,3404139178,10,8,83,http://images.amazon.com/images/P/3404139178.0...,Das Lacheln der Fortuna: Historischer Roman,10391,2622
3,276866,3404139178,9,8,11,http://images.amazon.com/images/P/3404139178.0...,Das Lacheln der Fortuna: Historischer Roman,10391,10886
4,106534,3404139178,6,8,6,http://images.amazon.com/images/P/3404139178.0...,Das Lacheln der Fortuna: Historischer Roman,10391,4228


In [ ]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [ ]:
users['Location'].value_counts()

london, england, united kingdom                  2506
toronto, ontario, canada                         2250
sydney, new south wales, australia               1744
melbourne, victoria, australia                   1708
portland, oregon, usa                            1629
                                                 ... 
gneisenaustr. 3, nordrhein-westfalen, germany       1
-, bayern, germany                                  1
bouchegouf, guelma, algeria                         1
indiana and philadelphia, pennsylvania, usa         1
tacoma, washington, united kingdom                  1
Name: Location, Length: 57339, dtype: int64

### Ищем id нужных книг

In [ ]:
for i, j in interactions[["product_id", "Book-Title"]].drop_duplicates().values:
    if "David Copperfield" in j:
        print("idx:", i, "\tBook Title:", j)

idx: 10168 	Book Title: David Copperfield (Wordsworth Classics)
idx: 1073 	Book Title: David Copperfield (Penguin Classics)


**Предобработка данных**

Удалим пользователей старше 85 лет, так как вряд-ли такие пользователи действительно оставляют комментарии и оценки книгам

In [ ]:
users_1 = users.loc[(users.Age <= 85)]
users_1

,User-ID,Location,Age
1,2,"stockton, california, usa",18.0
3,4,"porto, v.n.gaia, portugal",17.0
5,6,"santa monica, california, usa",61.0
9,10,"albacete, wisconsin, spain",26.0
10,11,"melbourne, victoria, australia",14.0
...,...,...,...
278848,278849,"georgetown, ontario, canada",23.0
278850,278851,"dallas, texas, usa",33.0
278851,278852,"brisbane, queensland, australia",32.0
278852,278853,"stranraer, n/a, united kingdom",17.0


Разобьем локацию на город, штат и страну

In [ ]:
import re
users_1 = users_1.loc[users['Location'].apply(lambda x: x.count(',')) == 2]
users_1.Location = users_1.Location.apply(lambda x: re.sub(r"[^A-Za-z,]+", '', x))
users_1 = users_1.join(users_1['Location'].str.replace(" ","").str.split(',',2,expand=True)).rename(columns={0:'City', 1:'State', 2:'Country'})
users_1

<ipython-input-466-e91ed6d5727c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users_1.Location = users_1.Location.apply(lambda x: re.sub(r"[^A-Za-z,]+", '', x))
<ipython-input-466-e91ed6d5727c>:4: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  users_1 = users_1.join(users_1['Location'].str.replace(" ","").str.split(',',2,expand=True)).rename(columns={0:'City', 1:'State', 2:'Country'})


,User-ID,Location,Age,City,State,Country
1,2,"stockton,california,usa",18.0,stockton,california,usa
3,4,"porto,vngaia,portugal",17.0,porto,vngaia,portugal
5,6,"santamonica,california,usa",61.0,santamonica,california,usa
9,10,"albacete,wisconsin,spain",26.0,albacete,wisconsin,spain
10,11,"melbourne,victoria,australia",14.0,melbourne,victoria,australia
...,...,...,...,...,...,...
278848,278849,"georgetown,ontario,canada",23.0,georgetown,ontario,canada
278850,278851,"dallas,texas,usa",33.0,dallas,texas,usa
278851,278852,"brisbane,queensland,australia",32.0,brisbane,queensland,australia
278852,278853,"stranraer,na,unitedkingdom",17.0,stranraer,na,unitedkingdom


In [ ]:
users.drop(columns=['Location'], inplace=True)

In [ ]:
interactions_sb = interactions.loc[interactions['User-ID'].isin(users['User-ID'])]

In [ ]:
interactions_sb['product_id'] = le.fit_transform(interactions_sb['ISBN'])
interactions_sb['vid'] = le.fit_transform(interactions_sb['User-ID'])

In [ ]:
interactions_sb.head()

,User-ID,ISBN,Book-Rating,user_num,books_num,picture_url,Book-Title,product_id,vid
0,86583,3404139178,9,8,18,http://images.amazon.com/images/P/3404139178.0...,Das Lacheln der Fortuna: Historischer Roman,10391,3445
1,132500,3404139178,10,8,43,http://images.amazon.com/images/P/3404139178.0...,Das Lacheln der Fortuna: Historischer Roman,10391,5248
2,66483,3404139178,10,8,83,http://images.amazon.com/images/P/3404139178.0...,Das Lacheln der Fortuna: Historischer Roman,10391,2622
3,276866,3404139178,9,8,11,http://images.amazon.com/images/P/3404139178.0...,Das Lacheln der Fortuna: Historischer Roman,10391,10886
4,106534,3404139178,6,8,6,http://images.amazon.com/images/P/3404139178.0...,Das Lacheln der Fortuna: Historischer Roman,10391,4228


Выберем страны, в которых более 30 пользователей

In [ ]:
resonable_countries = list(users_1.Country.value_counts()[(users_1.Country.value_counts()>30) & (users_1.Country.value_counts().index != '')].index)
len(resonable_countries)

73

In [ ]:
users_1.Country = users_1.Country.apply(lambda x: x if x in resonable_countries else 'others')
users_1

,User-ID,Location,Age,City,State,Country
1,2,"stockton,california,usa",18.0,stockton,california,usa
3,4,"porto,vngaia,portugal",17.0,porto,vngaia,portugal
5,6,"santamonica,california,usa",61.0,santamonica,california,usa
9,10,"albacete,wisconsin,spain",26.0,albacete,wisconsin,spain
10,11,"melbourne,victoria,australia",14.0,melbourne,victoria,australia
...,...,...,...,...,...,...
278848,278849,"georgetown,ontario,canada",23.0,georgetown,ontario,canada
278850,278851,"dallas,texas,usa",33.0,dallas,texas,usa
278851,278852,"brisbane,queensland,australia",32.0,brisbane,queensland,australia
278852,278853,"stranraer,na,unitedkingdom",17.0,stranraer,na,unitedkingdom


Для упрощения кластеризации создадим возрастные группы:

1. Дети
2. Подростки
3. Совершеннолетние
4. Взрослые

In [ ]:
age_groups = pd.DataFrame({'age_group': ['Childhood', 'Youth', 'Adulthood', 'Seniority'],
                           'start': [0., 9., 21., 45.],
                           'end': [8., 21., 45., 86.]})
age_groups

,age_group,start,end
0,Childhood,0.0,8.0
1,Youth,9.0,21.0
2,Adulthood,21.0,45.0
3,Seniority,45.0,86.0


In [ ]:
users_1

,User-ID,Location,Age,City,State,Country
1,2,"stockton,california,usa",18.0,stockton,california,usa
3,4,"porto,vngaia,portugal",17.0,porto,vngaia,portugal
5,6,"santamonica,california,usa",61.0,santamonica,california,usa
9,10,"albacete,wisconsin,spain",26.0,albacete,wisconsin,spain
10,11,"melbourne,victoria,australia",14.0,melbourne,victoria,australia
...,...,...,...,...,...,...
278848,278849,"georgetown,ontario,canada",23.0,georgetown,ontario,canada
278850,278851,"dallas,texas,usa",33.0,dallas,texas,usa
278851,278852,"brisbane,queensland,australia",32.0,brisbane,queensland,australia
278852,278853,"stranraer,na,unitedkingdom",17.0,stranraer,na,unitedkingdom


In [ ]:
age_groups.loc[(age_groups.start <= 0) & (age_groups.end >= 8), 'age_group'].iloc[0]

'Childhood'

In [ ]:
users_1['Age']

1         18.0
3         17.0
5         61.0
9         26.0
10        14.0
          ... 
278848    23.0
278850    33.0
278851    32.0
278852    17.0
278854    50.0
Name: Age, Length: 166694, dtype: float64

In [ ]:
age_groups_users = []

for age in tqdm.tqdm(users_1['Age']):
    age_groups_users.append(age_groups.loc[(age_groups.start <= age) & (age_groups.end >= age), 'age_group'].iloc[0])

users_1['Age'] = age_groups_users
users_clusters = users_1[['Age', 'Country']].drop_duplicates().reset_index(drop=True)
users_clusters.info()

100%|██████████| 166694/166694 [02:05<00:00, 1330.94it/s]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 274 entries, 0 to 273
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Age      274 non-null    object
 1   Country  274 non-null    object
dtypes: object(2)
memory usage: 4.4+ KB


Создадим функцию, которая будет выдавать картинки книг (в итоге оказалось, что ссылки протухшие и функция не используется)

In [ ]:
def get_chunk_imgs(l, n):
    for i in range(0, len(l), n):
        yield l[i:i + n]


def get_records_images(data):
    measure = data['Book-Title'].values
    top_n = 3
    picture_urls = data.picture_url.values
    cnt = 0
    for idx, chunk in enumerate(get_chunk_imgs(picture_urls, top_n)):
        fig = plt.figure(figsize=(20, 4))
        for n, i in enumerate(chunk):
            try:
                r = requests.get(i)
                im = Image.open(BytesIO(r.content))

            except:
                continue

            a = fig.add_subplot(1, top_n, n + 1)
            a.set_title("{}\nrecommendation number {}".format(measure[cnt], cnt+1), color='black')
            cnt += 1
            plt.imshow(im)
            plt.axis('off')
        plt.show()

def get_books_names(data):
    measure = data['Book-Title'].values
    top_n = 3


Задание 1.

In [ ]:
class TopNRecomendations():
    def __init__(self, clusters, users=users_1, data=interactions_sb, age_groups=age_groups, coutries=resonable_countries):
        self.clusters = clusters
        self.users = users
        self.data = data
        self.age_groups = age_groups
        self.countries = resonable_countries

    def get_book_score(self, data, b=2):
        max_user_num = data.user_num.max()
        max_rating = data['Book-Rating'].max()
        temp = data.groupby(by=['user_num', 'Book-Title'])['Book-Rating'].mean().reset_index()
        #Изначально предполагался вывод картинок, но к сожалению ссылки на изображения не работают
        temp = pd.merge(temp, data[['Book-Title', 'picture_url']].drop_duplicates(subset='Book-Title'), how='inner', on='Book-Title')
        temp['Book-Rating'] = temp['Book-Rating'] / max_rating
        temp['user_num'] = temp['user_num'] / max_user_num
        temp = temp.assign(score = (temp['Book-Rating'] + temp['user_num']) / 2).sort_values(by='score', ascending=False)
        return temp

    def top_items(self, country, age, top_count=10):
        country = country if country in self.countries else 'others'
        age = self.age_groups.loc[(self.age_groups.start <= age) & (self.age_groups.end >= age), 'age_group'].iloc[0]
        suitable_users = self.users.loc[(self.users.Age == age) & (self.users.Country == country), 'User-ID']
        suitable_data = pd.merge(self.data, suitable_users, how='inner', on='User-ID')
        book_score = self.get_book_score(suitable_data)

        get_books_names(book_score[:top_count])
        return book_score[:top_count]

a = TopNRecomendations(clusters=users_clusters)
check = a.top_items(country='usa', age=35.)

In [ ]:
check

,user_num,Book-Title,Book-Rating,picture_url,score
9013,1.000000,The Lovely Bones: A Novel,0.835762,http://images.amazon.com/images/P/0316666343.0...,0.917881
9011,0.688826,The Da Vinci Code,0.851613,http://images.amazon.com/images/P/0385504209.0...,0.770219
9010,0.541726,The Red Tent (Bestselling Backlist),0.844186,http://images.amazon.com/images/P/0312195516.0...,0.692956
9008,0.442716,Harry Potter and the Sorcerer's Stone (Harry P...,0.909524,http://images.amazon.com/images/P/059035342X.0...,0.676120
8390,0.434229,The Secret Life of Bees,0.844186,http://images.amazon.com/images/P/0670032379.0...,0.639208
9009,0.452617,Divine Secrets of the Ya-Ya Sisterhood: A Novel,0.814130,http://images.amazon.com/images/P/0060928336.0...,0.633374
9007,0.417256,Where the Heart Is (Oprah's Book Club (Paperba...,0.837647,http://images.amazon.com/images/P/0446672211.0...,0.627452
6860,0.302687,To Kill a Mockingbird,0.930000,http://images.amazon.com/images/P/0446310786.0...,0.616344
8999,0.291372,Harry Potter and the Order of the Phoenix (Boo...,0.922973,http://images.amazon.com/images/P/043935806X.0...,0.607172
9012,0.821782,Wild Animus,0.389394,http://images.amazon.com/images/P/0971880107.0...,0.605588


In [ ]:
a.top_items(country='spain', age=35)

,user_num,Book-Title,Book-Rating,picture_url,score
518,1.000000,The Lovely Bones: A Novel,0.750000,http://images.amazon.com/images/P/0316666343.0...,0.875000
516,0.688826,The Da Vinci Code,0.833333,http://images.amazon.com/images/P/0385504209.0...,0.761080
515,0.434229,The Secret Life of Bees,1.000000,http://images.amazon.com/images/P/0142001740.0...,0.717115
512,0.380481,Angels &amp; Demons,1.000000,http://images.amazon.com/images/P/0671027360.0...,0.690240
507,0.294201,The Firm,1.000000,http://images.amazon.com/images/P/044021145X.0...,0.647100
517,0.821782,Wild Animus,0.450000,http://images.amazon.com/images/P/0971880107.0...,0.635891
501,0.227723,The Hobbit : The Enchanting Prelude to The Lor...,1.000000,http://images.amazon.com/images/P/0345339681.0...,0.613861
509,0.302687,To Kill a Mockingbird,0.900000,http://images.amazon.com/images/P/0446310786.0...,0.601344
508,0.302687,Summer Sisters,0.900000,http://images.amazon.com/images/P/0440226430.0...,0.601344
495,0.189533,Cold Mountain : A Novel,1.000000,http://images.amazon.com/images/P/0375700757.0...,0.594767


Задание 2

In [ ]:
books_meets

,ISBN,user_num
0,0330299891,1
1,0375404120,1
2,9022906116,1
3,#6612432,1
4,'9607092910',1
...,...,...
185968,"\8888809228\""""",1
185969,"\9170010242\""""",1
185970,ooo7156103,1
185971,´3499128624,1


In [ ]:
user_meets

,User-ID,books_num
0,8,7
1,9,1
2,10,1
3,12,1
4,14,3
...,...,...
77800,278846,1
77801,278849,1
77802,278851,14
77803,278852,1


In [ ]:
interactions_clast = interactions.merge(books_meets, on=["ISBN"]).merge(user_meets, on=["User-ID"])
interactions_clast

,User-ID,ISBN,Book-Rating,user_num_x,books_num_x,picture_url,Book-Title,product_id,vid,user_num_y,books_num_y
0,86583,3404139178,9,8,18,http://images.amazon.com/images/P/3404139178.0...,Das Lacheln der Fortuna: Historischer Roman,10391,3445,8,18
1,86583,3453061187,8,13,18,http://images.amazon.com/images/P/3453061187.0...,Die Jury. Roman.,10488,3445,13,18
2,86583,3426609061,7,8,18,http://images.amazon.com/images/P/3426609061.0...,House Of God.,10422,3445,8,18
3,132500,3404139178,10,8,43,http://images.amazon.com/images/P/3404139178.0...,Das Lacheln der Fortuna: Historischer Roman,10391,5248,8,43
4,132500,3499232871,8,11,43,http://images.amazon.com/images/P/3499232871.0...,Herzsprung,10544,5248,11,43
...,...,...,...,...,...,...,...,...,...,...,...
110489,24172,0590483404,8,9,20,http://images.amazon.com/images/P/0590483404.0...,"A Shocker on Shock Street (Goosebumps, No 35)",7074,879,9,20
110490,190082,0525460543,7,6,6,http://images.amazon.com/images/P/0525460543.0...,The Prince of Egypt Collector's Edition Storybook,6206,7498,6,6
110491,206652,1860465811,9,8,10,http://images.amazon.com/images/P/1860465811.0...,Wind-Up Bird Chronicle,10203,8112,8,10
110492,223525,1588720284,8,6,13,http://images.amazon.com/images/P/1588720284.0...,Why Your Life Sucks: And What You Can Do Abou...,10132,8754,6,13


In [ ]:
books_transform = preprocessing.LabelEncoder()
users_transform = preprocessing.LabelEncoder()
interactions_clast["product_id"] = books_transform.fit_transform(interactions_clast["Book-Title"])
interactions_clast["vid"] = users_transform.fit_transform(interactions_clast["User-ID"])

Составим спарс матрицу, так как в оперативную память полная матрица не помещается

In [ ]:
csr_rates = coo_matrix((interactions_clast["Book-Rating"], (interactions_clast["vid"], interactions_clast["product_id"])),
                            shape=(len(set(interactions_clast["vid"])), len(set(interactions_clast["product_id"]))))


Сделаем кластеризацию при помощи DBSCAN, метрикой будет косинусное расстояние

In [ ]:
from sklearn.cluster import DBSCAN

dbscan = DBSCAN(eps=0.8, min_samples=3, metric='cosine')
dbscan.fit(csr_rates)

DBSCAN(eps=0.8, metric='cosine', min_samples=3)

In [ ]:
interactions_clast

,User-ID,ISBN,Book-Rating,user_num_x,books_num_x,picture_url,Book-Title,product_id,vid,user_num_y,books_num_y
0,86583,3404139178,9,8,18,http://images.amazon.com/images/P/3404139178.0...,Das Lacheln der Fortuna: Historischer Roman,1734,3445,8,18
1,86583,3453061187,8,13,18,http://images.amazon.com/images/P/3453061187.0...,Die Jury. Roman.,1966,3445,13,18
2,86583,3426609061,7,8,18,http://images.amazon.com/images/P/3426609061.0...,House Of God.,3269,3445,8,18
3,132500,3404139178,10,8,43,http://images.amazon.com/images/P/3404139178.0...,Das Lacheln der Fortuna: Historischer Roman,1734,5248,8,43
4,132500,3499232871,8,11,43,http://images.amazon.com/images/P/3499232871.0...,Herzsprung,3147,5248,11,43
...,...,...,...,...,...,...,...,...,...,...,...
110489,24172,0590483404,8,9,20,http://images.amazon.com/images/P/0590483404.0...,"A Shocker on Shock Street (Goosebumps, No 35)",275,879,9,20
110490,190082,0525460543,7,6,6,http://images.amazon.com/images/P/0525460543.0...,The Prince of Egypt Collector's Edition Storybook,7988,7498,6,6
110491,206652,1860465811,9,8,10,http://images.amazon.com/images/P/1860465811.0...,Wind-Up Bird Chronicle,9282,8112,8,10
110492,223525,1588720284,8,6,13,http://images.amazon.com/images/P/1588720284.0...,Why Your Life Sucks: And What You Can Do Abou...,9246,8754,6,13


In [ ]:
np.unique(dbscan.labels_)

array([-1,  0])

In [ ]:
class UserClustering():
    def __init__(self, data=interactions_clast):
        self.data = data

    def get_book_score(self, data, b=2):
        max_user_num = data.user_num_x.max()
        max_rating = data['Book-Rating'].max()
        temp = data.groupby(by=['user_num_x', 'Book-Title'])['Book-Rating'].mean().reset_index()
        temp = pd.merge(temp, data[['Book-Title', 'picture_url']].drop_duplicates(subset='Book-Title'), how='inner', on='Book-Title')
        temp['Book-Rating'] = temp['Book-Rating'] / max_rating
        temp['user_num'] = temp['user_num_x'] / max_user_num
        temp = temp.assign(score = (1 + b**2) * (temp['Book-Rating'] * temp['user_num'])/ ((b**2 * temp['user_num']) + temp['Book-Rating'])).sort_values(by='score', ascending=True)
        return temp

    def top_items(self, users, top_count=10):
        suitable_data = interactions_clast.loc[interactions_clast['vid'].isin(users)]
        book_score = self.get_book_score(suitable_data)
        return book_score[:top_count]

In [ ]:
for clust in [-1, 0]:
    a = UserClustering()
    check = a.top_items(users=np.where(dbscan.labels_ == clust)[0])
    for item in check['Book-Title']:
        print(item)

    print('-----------------------------------')

Stealing Time (April Woo Suspense Novels (Paperback))
Lambs of God
Fear and Loathing: On the Campaign Trail
22 Indigo Place
Slightly Shady
Stranger in My Arms
Out Of The Shadows (Love Inspired (Numbered))
One True Love? (Harlequin Flipside, No. 2)
Driving Mr. Albert: A Trip Across America with Einstein's Brain
Summer of My German Soldier
-----------------------------------
Desert Rain
The Landry News
The Body Artist: A Novel
Innkeeping With Murder (Lighthouse Inn Mysteries)
Doctor'S Daughter  (Men Of Glory) (Harlequin Superromance, 835)
Fire
The Miracles of Santo Fico
Faserland
Wreck the Halls
Die LÃ?Â¼ge im Bett.
-----------------------------------


In [ ]:
check

,user_num_x,Book-Title,Book-Rating,picture_url,user_num,score
2501,7,Desert Rain,0.10,http://images.amazon.com/images/P/0380767627.0...,0.009901,0.035461
1741,6,The Landry News,0.20,http://images.amazon.com/images/P/0689828683.0...,0.008487,0.036276
1549,6,The Body Artist: A Novel,0.25,http://images.amazon.com/images/P/0743203968.0...,0.008487,0.037360
805,6,Innkeeping With Murder (Lighthouse Inn Mysteries),0.30,http://images.amazon.com/images/P/0425180026.0...,0.008487,0.038119
443,6,Doctor'S Daughter (Men Of Glory) (Harlequin S...,0.30,http://images.amazon.com/images/P/0373708351.0...,0.008487,0.038119
577,6,Fire,0.30,http://images.amazon.com/images/P/0393010465.0...,0.008487,0.038119
1785,6,The Miracles of Santo Fico,0.30,http://images.amazon.com/images/P/0446531030.0...,0.008487,0.038119
555,6,Faserland,0.30,http://images.amazon.com/images/P/3442433614.0...,0.008487,0.038119
2170,6,Wreck the Halls,0.35,http://images.amazon.com/images/P/0553582267.0...,0.008487,0.038681
420,6,Die LÃ?Â¼ge im Bett.,0.35,http://images.amazon.com/images/P/349222539X.0...,0.008487,0.038681


Задание 3

In [ ]:
class Recomendations():
    def __init__(self, Int):
        self.interactions = Int


    def fit(self):
        Int = self.interactions[["vid", "product_id", "Book-Rating"]].drop_duplicates()
        user_products = Int.groupby(["vid"])["product_id"].apply(list).reset_index()
        user_rates = Int.groupby(["vid"])["Book-Rating"].apply(list).reset_index()
        user_products['Rates'] = user_rates["Book-Rating"]
        product_num = [len(i) for i in user_products["product_id"]]
        user_products["prod_num"] = product_num
        user_products = user_products[user_products["prod_num"] > 1]
        cooc = {}
        for i in tqdm.tqdm_notebook(user_products.values):
            for j in range(len(i[1])):
                for k in range(len(i[1])):
                    if j != k:
                        try:
                            cooc[str(i[1][j]) + "_" + str(i[1][k])] += (i[2][j] + i[2][k]) / 2
                        except:
                            cooc[str(i[1][j]) + "_" + str(i[1][k])] = (i[2][j] + i[2][k]) / 2
        cooc_list = []
        for i, j in cooc.items():
            if j != 1:
                cooc_list.append(i.split("_") + [j])
        self.cooc_rec = pd.DataFrame(cooc_list, columns=["item1", "item2", "measure"])

    def predict(self, i, show=False):
        recs = self.cooc_rec[self.cooc_rec["item1"] == str(i)]\
                            .sort_values("measure", ascending=False)\
                            .head(10)
        print(u"Для товара")
        print(interactions[interactions['product_id'] == i]['Book-Title'].iloc[0])
        print()
        print(u"Такие рекомендации")
        for idx in recs["item2"].values.astype(int):
            print(interactions[interactions['product_id'] == idx]['Book-Title'].iloc[0])

In [ ]:
cooc_rec = Recomendations(interactions)
cooc_rec.fit()

<ipython-input-516-c3dba52e9b90>:19: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm.tqdm_notebook(user_products.values):


  0%|          | 0/10011 [00:00<?, ?it/s]

In [ ]:
cooc_rec.predict(867)

Для товара
Love in the Time of Cholera

Такие рекомендации
Notes From a Small Planet
Chocolat
Wuthering Heights (Wordsworth Classics)
Wild Animus
Emma (Penguin Popular Classics)
The Diving Bell and the Butterfly : A Memoir of Life in Death
Pride and Prejudice (Oxford World's Classics)
Lucy Sullivan Is Getting Married
Dracula (Penguin Classics)
Mr. Maybe


Задание 4

In [ ]:
class Collaborative():
    def __init__(self, interactions):
        interactions.loc[interactions['Book-Rating'] < 5, 'weight'] = 0
        interactions.loc[interactions['Book-Rating'] >= 5, 'weight'] = 1
        weighted = interactions[["vid", "product_id", "picture_url", "weight"]]
        weighted["weight"] = (weighted["weight"] > 0).astype(int)
        self.csr_rates = coo_matrix((weighted["weight"], (weighted["vid"], weighted["product_id"])),
                            shape=(len(set(weighted["vid"])), len(set(weighted["product_id"]))))


    def user_based(self, idx):
        user_rates = self.csr_rates.getrow(idx).toarray()[0]
        watched_items = np.where(user_rates != 0)[0]
        metrics = cosine_similarity([user_rates], self.csr_rates).reshape(-1, 1)
        rates = csr_matrix.multiply(self.csr_rates.copy(), metrics)
        total_rate = (1 - user_rates.astype(bool)) * np.array(np.sum(rates, axis=0))[0]
        self.get_rec(watched_items, np.arange(self.csr_rates.shape[1])[np.argsort(total_rate)[::-1][:10]],
                    np.sort(total_rate)[::-1][:10])

    def item_based(self, idx):
        item_rates = self.csr_rates.getcol(idx).toarray().reshape(1, -1)
        metrics = cosine_similarity(item_rates, self.csr_rates.T).reshape(-1, 1)
        rates = csr_matrix.multiply(self.csr_rates.T.copy(), metrics)
        total_rate = np.array(np.sum(rates, axis=1)).ravel()
        self.get_rec([idx], np.arange(self.csr_rates.shape[1])[np.argsort(total_rate)[::-1][:10]],
                    np.sort(total_rate)[::-1][:10])

    def get_rec(self, watched, recs, measure):
        print(u"Для таких товаров\n")
        for i in watched:
            print(interactions[interactions['product_id'] == i]['Book-Title'].iloc[0])
        print(u"Такие рекомендации\n")
        for i in recs:
            print(interactions[interactions['product_id'] == i]['Book-Title'].iloc[0])
        print(measure)

In [ ]:
clb = Collaborative(interactions)

<ipython-input-536-9774226dbbc0>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weighted["weight"] = (weighted["weight"] > 0).astype(int)


In [ ]:
interactions

,User-ID,ISBN,Book-Rating,user_num,books_num,picture_url,Book-Title,product_id,vid,weight
0,86583,3404139178,9,8,18,http://images.amazon.com/images/P/3404139178.0...,Das Lacheln der Fortuna: Historischer Roman,10391,3445,1.0
1,132500,3404139178,10,8,43,http://images.amazon.com/images/P/3404139178.0...,Das Lacheln der Fortuna: Historischer Roman,10391,5248,1.0
2,66483,3404139178,10,8,83,http://images.amazon.com/images/P/3404139178.0...,Das Lacheln der Fortuna: Historischer Roman,10391,2622,1.0
3,276866,3404139178,9,8,11,http://images.amazon.com/images/P/3404139178.0...,Das Lacheln der Fortuna: Historischer Roman,10391,10886,1.0
4,106534,3404139178,6,8,6,http://images.amazon.com/images/P/3404139178.0...,Das Lacheln der Fortuna: Historischer Roman,10391,4228,1.0
...,...,...,...,...,...,...,...,...,...,...
110489,27551,0849912970,10,6,6,http://images.amazon.com/images/P/0849912970.0...,Traveling Light: Releasing the Burdens You Wer...,9472,1035,1.0
110490,206652,1860465811,9,8,10,http://images.amazon.com/images/P/1860465811.0...,Wind-Up Bird Chronicle,10203,8112,1.0
110491,223525,1588720284,8,6,13,http://images.amazon.com/images/P/1588720284.0...,Why Your Life Sucks: And What You Can Do Abou...,10132,8754,1.0
110492,94153,0060652926,9,7,14,http://images.amazon.com/images/P/0060652926.0...,Mere Christianity,275,3740,1.0


Для данной книги рекомендует вроде бы на похожем языке => рекомендации похожи на правду. Гугл данных произведений показал, что они достаточно похожи

In [ ]:
clb.item_based(10391)

Для таких товаров

Das Lacheln der Fortuna: Historischer Roman
Такие рекомендации

Das Lacheln der Fortuna: Historischer Roman
Der Vorleser
Die PÃ?Â¤pstin.
Harry Potter und der Stein der Weisen
Harry Potter und die Kammer des Schreckens
Die unendliche Geschichte: Von A bis Z
Harry Potter und der Gefangene von Azkaban
Harry Potter Und Der Feuerkelch
Die Jury. Roman.
The Physician
[6.         1.63299316 1.41421356 1.3540064  1.29099445 1.29099445
 1.22474487 1.15470054 1.08012345 1.08012345]


По пользователям рекомендации тоже выглядят адекватно

In [ ]:
clb.user_based(5041)

Для таких товаров

Five Quarters of the Orange
If on a Winter's Night a Traveler
I Think I Love You
Where You Belong
The Best Laid Plans
Midnight in the Garden of Good and Evil: A Savannah Story
Four Blondes
You Shall Know Our Velocity
Такие рекомендации

The Lovely Bones: A Novel
Girl with a Pearl Earring
Harry Potter and the Order of the Phoenix (Book 5)
The Red Tent (Bestselling Backlist)
The Secret Life of Bees
The Poisonwood Bible: A Novel
SHIPPING NEWS
Life of Pi
Angels &amp; Demons
A Heartbreaking Work of Staggering Genius
[1.57739217 0.91722074 0.69037477 0.66970817 0.622493   0.58621675
 0.58492832 0.58424749 0.57016205 0.56396579]
